In [1]:
!tree -d -L 2 ~/datasets/data/cityscapes

/home/skywatcher/datasets/data/cityscapes
├── gtCoarse
│   ├── train
│   ├── train_extra
│   └── val
├── gtFine
│   ├── test
│   ├── train
│   └── val
└── leftImg8bit
    ├── test
    ├── train
    ├── train_extra
    └── val

13 directories


In [2]:
#Let's do some data preprocessing

#Import the packages
import os
import glob
from shutil import copy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

'''
Given a two lists of paths to images and masks, and two destination directories
for the images and masks.
'''
def cs_create_copies(images, masks, image_dest, mask_dest, mask_only=False):
    total = len(images)
    image_count = 0
    #For every image-mask pair
    for image, mask in zip(images, masks):
        #Get the filename we'll use
        filename = image.split('/')[-1]
        filename = filename[:filename.rindex('_')] + '.png'
        #Create the symbolic links
        if not mask_only:
            os.symlink(image, image_dest + '/' + filename)
        os.symlink(mask, mask_dest + '/' + filename)
        image_count+=1
        percentage = image_count/total*100
        print('Copying Images and Masks: %d%%\r'%(percentage),end="")

In [3]:
'''
A utility function to match two lists of files, mainly used in 
cs_preprocessing_files to ensure that the recieved file names from glob
match after each list has been sorted. This ensures that each image list is in the
same order as its corresponding mask list. In the case its not, then something has gone
wrong with the importing of files into the cityscapes directory.

@param images - A list of image files, when used in cs_preprocessing_files it will be absolute paths to the files
@param masks - A list of mask files, like images will be absolute when used in cs_preprocessing_files
'''
def cs_match_files(images, masks):
    #If the two lists are of equal size
    if len(images) == len(masks):
        #Go through file by file in each list
        for image, mask in zip(images, masks):
            #Get the sequence for cityscapes
            filename = image.split('/')[-1]
            match_to = filename[:filename.rindex('_')]
            #If the sequence isn't matching to the current mask then return False
            if match_to not in mask:
                print("The list hasn't been sorted properly")
                return False
    #Lists don't have the same size! return False.
    else:
        print("Lengths of the images and masks list do not match!")
        return False
    #Reaches here if all is good, so return True
    return True

In [4]:
'''
The task of this function will be to take the extracted cityscapes data
given by source, which I will assume is a directory containing the following
structure:
cityscapes
├── gtCoarse
│   ├── train
│   ├── train_extra
│   └── val
├── gtFine
│   ├── test
│   ├── train
│   └── val
└── leftImg8bit
    ├── test
    ├── train
    ├── train_extra
    └── val

This will be the structure if you downloaded the cityscapes dataset and extracted it.
The datasets you got should have been the leftImg8bit, leftImg8bit_trainextra, gtFine 
and gtCoarse.

@param source - the directory for cityscapes that has the structure mentioned above (This is best done as an absolute path)

'''
def cs_preprocessing_files(source):
    #Create all the variables
    fine_masks_dir = source + '/train/masks//all'
    coarse_masks_dir = source + '/train/coarse_masks/all'
    fine_train_dir = source + '/train/images/all'
    coarse_train_dir = source + '/train/coarse_images/all'
    val_fine_masks_dir = source + '/val/masks/all'
    val_coarse_masks_dir = source + '/val/coarse_masks/all'
    val_images_dir = source + '/val/images/all'
    
    #Create all the directories
    os.makedirs(fine_masks_dir, exist_ok=True)
    os.makedirs(coarse_masks_dir, exist_ok=True)
    os.makedirs(fine_train_dir, exist_ok=True)
    os.makedirs(coarse_train_dir, exist_ok=True)
    os.makedirs(val_fine_masks_dir, exist_ok=True)
    os.makedirs(val_coarse_masks_dir, exist_ok=True)
    os.makedirs(val_images_dir, exist_ok=True)
    
    #Get all the paths of the images and sort them
    fine_images = glob.glob(source+'/leftImg8bit/train/*/*.png')
    fine_images.sort()
    coarse_images = fine_images + glob.glob(source + '/leftImg8bit/train_extra/*/*.png')
    coarse_images.sort()
    #Get all the masks and sort them
    fine_masks = glob.glob(source+'/gtFine/train/*/*color*.png')
    fine_masks.sort()
    coarse_masks = glob.glob(source+'/gtCoarse/train*/*/*color*.png')
    coarse_masks.sort()
    
    #Get all the validation paths and sort them
    val_images = glob.glob(source+'/leftImg8bit/val/*/*.png')
    val_images.sort()
    val_fine_masks = glob.glob(source+'/gtFine/val/*/*color*.png')
    val_fine_masks.sort()
    val_coarse_masks = glob.glob(source+'/gtCoarse/val/*/*color*.png')
    val_coarse_masks.sort()
    
    #Now verify all the paths have been retrieved and are in order
    if cs_match_files(fine_images, fine_masks):
        print('{} Fine Images and Masks Detected and in order'.format(len(fine_images)))
    if cs_match_files(fine_images, fine_masks):
        print('{} Coarse Images and Masks Detected and in order'.format(len(coarse_images)))
    if cs_match_files(val_images, val_fine_masks):
        print('{} Val Fine Images and Masks Detected and in order'.format(len(val_images)))
    if cs_match_files(val_images, val_coarse_masks):
        print('{} Val Coarse Images and Masks Detected and in order'.format(len(val_images)))
    
    #Now starting with the fine images, create the copies
    print("Copying the Fine Images and Masks")
    cs_create_copies(fine_images, fine_masks, fine_train_dir, fine_masks_dir)
    print("Copying the Fine Validation Images")
    cs_create_copies(val_images, val_fine_masks, val_images_dir, val_fine_masks_dir)
    print("Copying the Coarse Images and Masks")
    cs_create_copies(coarse_images, coarse_masks, coarse_train_dir, coarse_masks_dir)
    print("Copying the Coarse Validation Images and Masks")
    cs_create_copies(val_images, val_coarse_masks, val_images_dir, val_coarse_masks_dir, mask_only=True)

In [23]:
'''
An ImageDataGenerator will be created using keras ready for use in training TF2.0 models
like FastSCNN. 

@param images - The directory where all the training images are kept
@param masks - The directory where all the masks are kept
@param val_images - The directory where all validation images are kept
@param val_masks - The directory where all validation masks are kept
@param target_size - Should be a tuple of two numbers, the image size needed for the model should be put here. 
                     This will be largely dependent on the network you are training.
@return ImageDataGenerator - The ImageDataGenerator
'''
def cs_create_generators(images, masks, val_images, val_masks, target_size):
    
    # we create two instances with the same arguments
    data_gen_args = dict(rotation_range=90,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         zoom_range=0.2,
                         rescale=1/255)
    
    image_datagen = ImageDataGenerator(**data_gen_args)
    val_image_datagen = ImageDataGenerator()
    mask_datagen = ImageDataGenerator(**data_gen_args)
    val_mask_datagen = ImageDataGenerator()
    
    # Provide the same seed to the flow methods
    seed = 42

    #Create the generators
    image_generator = image_datagen.flow_from_directory(
        images,
        class_mode=None,
        follow_links=True,
        target_size=target_size,
        seed=seed)

    mask_generator = mask_datagen.flow_from_directory(
        masks,
        class_mode=None,
        follow_links=True,
        target_size=target_size,
        seed=seed)
    
    #Create the generators
    val_image_generator = val_image_datagen.flow_from_directory(
        val_images,
        class_mode=None,
        follow_links=True,
        target_size=target_size,
        seed=seed)

    val_mask_generator = val_mask_datagen.flow_from_directory(
        val_masks,
        class_mode=None,
        follow_links=True,
        target_size=target_size,
        seed=seed)

    # combine generators into one which yields image and masks
    train_generator = (pair for pair in zip(image_generator, mask_generator))
    val_generator = (pair for pair in zip(val_image_generator, val_mask_generator))
    return train_generator, val_generator

In [6]:
cs_preprocessing_files('/home/skywatcher/datasets/data/cityscapes')

2975 Fine Images and Masks Detected and in order
22973 Coarse Images and Masks Detected and in order
500 Val Fine Images and Masks Detected and in order
500 Val Coarse Images and Masks Detected and in order
Copying the Fine Images and Masks
Copying the Fine Validation Images
Copying the Coarse Images and Masks
Copying the Coarse Validation Images and Masks


In [7]:
!tree -d -L 2 /home/skywatcher/datasets/data/cityscapes

/home/skywatcher/datasets/data/cityscapes
├── gtCoarse
│   ├── train
│   ├── train_extra
│   └── val
├── gtFine
│   ├── test
│   ├── train
│   └── val
├── leftImg8bit
│   ├── test
│   ├── train
│   ├── train_extra
│   └── val
├── train
│   ├── coarse_images
│   ├── coarse_masks
│   ├── images
│   └── masks
└── val
    ├── coarse_masks
    ├── images
    └── masks

22 directories


In [24]:
images = '/home/skywatcher/datasets/data/cityscapes/train/images'
masks = '/home/skywatcher/datasets/data/cityscapes/train/masks'
val_images = '/home/skywatcher/datasets/data/cityscapes/val/images'
val_masks = '/home/skywatcher/datasets/data/cityscapes/val/masks'
train_generator, val_generator = cs_create_generators(images, masks, val_images, val_masks, (2048, 1024))

Found 2975 images belonging to 1 classes.
Found 2975 images belonging to 1 classes.
Found 500 images belonging to 1 classes.
Found 500 images belonging to 1 classes.


In [25]:
import fast_scnn
fastscnn = fast_scnn.create_fastscnn()
optimizer = tf.keras.optimizers.SGD(momentum=0.9, lr=0.045)
fastscnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [26]:
fastscnn.fit_generator(
        train_generator,
        epochs=5,
        steps_per_epoch=170,
        validation_data=val_generator,
        validation_steps=32)

Epoch 1/5


ValueError: A target array with shape (32, 2048, 1024, 3) was passed for an output of shape (None, 2048, 1024, 19) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [ ]:
fastscnn.summary()

In [ ]:
tf.keras.utils.plot_model(fastscnn, show_layer_names=True, show_shapes=True)